# 인스타그램 계정별로 크롤링하기

* 전달받은 데이터셋 : 영화홍보 or 영화 제작사 등 특정 영화의 공식 인스타그램 계정 아이디가 있는 데이터
* 작업 목적 : 당사의 어플('마이무비')에서 해당 영화의 인스타그램 피드 정보를 추가적으로 제공함

In [ ]:
import pymysql
import pandas as pd
from datetime import datetime
import urllib
import requests
import json
import os
import re
import pyquery
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import random

today = datetime.today().strftime("%m%d")


def db_connection(host_name='ds'): 
    
    host_url = ""
    user_nm = ""
    passwd = ""
    port_num = 
    db_name = ""
    
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8', db = db_name,cursorclass=pymysql.cursors.DictCursor) # cursorclass=pymysql.cursors.DictCursor 추가     # db=db,
    
    return conn

def at(x):
    x = str(x)
    x = x.replace("@","")
    x = x.replace("=","")
    return x

def get_web_html(url):

    global rest
    rest = random.randint(10,12)
    try:
        response = requests.get(url, headers=headers)
        time.sleep(rest)
        # http 응답 상태 코드로 확인
        if response.status_code == 200: # 서버에 요청한 페이지가 있을 때
            return response.text
        else:
            print('Wrong：', response.status_code)
    except Exception as e:
        print(e)
        return None

def blank(x):
    x = x.strip()
    x = x.replace('\r','')
    
    return x

def strange_r(x):
    
    x = x.replace('\r','')
    
    return x


title_sql = '''
select 칼럼명 from movie_data;
'''

instagram_sql = '''
select 칼럼명
where instagram_id is not null;

'''


cursor = db_connection().cursor()
cursor.execute(title_sql)
title_list = pd.DataFrame(cursor.fetchall())
time.sleep(1)

cursor.execute(instagram_sql)
insta_list = pd.DataFrame(cursor.fetchall())



insta_list['instagram_id'] = insta_list['instagram_id'].apply(at)

crawl_list = insta_list.merge(title_list, on=['기준칼럼들'], how='left')

crawl_list = crawl_list[crawl_list['instagram_id'] != 'www.instagram.com']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'instagram.com']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'explore']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'p']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'www.instagram']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'Instagram']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'Instagram.com']
crawl_list = crawl_list[crawl_list['instagram_id'] != 'instagram']


error_list = []
data_list = []
ommision_list = []


# 계정 아이디가 들어있는 데이터 프레임을 한 줄씩 보냄
for idx,movie_list in tqdm(crawl_list[2500:-1].iterrows(), desc="돌리는중"):
    cnt = 0
    기준칼럼1 = movie_list['']
    기준칼럼2 = movie_list['']
    instagram_id = movie_list['instagram_id']
    title = movie_list['title']
    기준칼럼3 = movie_list['']
    기준칼럼4 = movie_list['']


    print(instagram_id) 
    url = 'https://www.instagram.com/' + str(movie_list['instagram_id'])
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    }
    html = get_web_html(url)

    try : 
        user_id = re.findall('profilePage_([0-9]+)', html, re.S)[0]
        doc = pyquery.PyQuery(html)
        items = doc('script[type="text/javascript"]').items()
        # query latest 12 post
        for item in items:
            if item.text().strip().startswith('window._sharedData'):
                js_data = json.loads(item.text()[21:-1], encoding='utf-8')
                edges = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']
                cursor = \
                js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info'][
                    'end_cursor']
                flag = js_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
                
                
                
                for edge in edges:
                    shortcode = edge['node']['shortcode']
                    peed_url = 'https://www.instagram.com/p/' + shortcode
                    img_url = peed_url + '/media/?size=l'
                    cnt = cnt + 1
                    #print(cnt)
                    
                    if cnt == 1 :
                        peed_post = 'most recent post of '+ str(title) +'\'s official instagram page'
                    if cnt == 2 :
                        peed_post = '2nd recent post of '+ str(title)  +'\'s official instagram page'
                    if cnt == 3 :
                        peed_post = '3rd recent post of '+ str(title)  +'\'s official instagram page'
                        
                    elif  3 < cnt < 11 :
                        peed_post = str(cnt) +'th recent post of ' + str(title) +'\'s offical instagram page'
                    
                    else : 
                        pass
                    
                    data = {
                            "기준칼럼1" :기준칼럼1,
                            "기준칼럼2":기준칼럼2,
                            "기준칼럼3":기준칼럼3,
                            "기준칼럼4" : 기준칼럼4,
                            "instagram_id": instagram_id,
                            "img_url" : img_url, 
                            "peed_post" : peed_post,
                            "peed_url" : peed_url
                           }
                    
                    if cnt < 11 :
                        data_list.append(data)   
                
                # get_twelve_post(edges)
                # 계정은 있으나 비공개 계정이거나 게시글이 아예 없을 때
                if (flag == False) & (len(edges) == 0):
                    ommision = {"instagram_id":instagram_id}
                    ommision_list.append(ommision)
                else : 
                    pass

    except :
        # url 정보를 가져올 수 없을 때 (계정이 없는 경우임. 페이지 없음)---> 예외처리
        error = {"instagram_id":instagram_id}
        error_list.append(error)
        print('error page - !')

    if (idx+1) % 50 == 0 :
        time.sleep(50)
    else :
        pass  

result_df = pd.DataFrame(data_list)
error_df = pd.DataFrame(error_list)
ommision_df = pd.DataFrame(ommision_list)


result_df['peed_url'] = result_df['peed_url'].apply(blank)
result_df['img_url'] = result_df['img_url'].apply(blank)
result_df['peed_post'] = result_df['peed_post'].apply(strange_r)

# 정상 크롤링 데이터
# url 오류로 엑셀 변환시 데이터삽입안되서 다음과 같은 옵션 추가해야함
url_preserve = pd.ExcelWriter(f'/Users/myhellebs/Desktop/maimovie_instagram/maimovie_insta_crawling_result_{today}.xlsx', options={'strings_to_urls': False})
result_df.to_excel(url_preserve)
url_preserve.save()
# 계정 존재하지 않는 에러 데이터 
error_df.to_excel(f"/Users/myhellebs/Desktop/maimovie_instagram/maimovie_insta_crawling_error_{today}.xlsx")
# 비공개 계정 or 게시글 없는 계정 데이터
ommision_df.to_excel(f"/Users/myhellebs/Desktop/maimovie_instagram/maimovie_insta_crawling_none_{today}.xlsx")